# Importing Packages

In [61]:
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model #GARCH Models
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
import math
import scipy.stats
from statistics import NormalDist
from scipy.stats import t
from scipy.stats import f
from datetime import datetime
from scipy.stats import shapiro #Shapiro Test
from scipy import stats #t-test
import statsmodels.api as sm #Ljung-Box Test
from scipy.stats import chi2

# Price and Returns

In [62]:
start_date = '2006-01-01' #yyyy-mm-dd
end_date = '2021-01-01'


# Singapore
SG = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/2006-2021/STI.csv')
SG['Date'] = pd.to_datetime(SG['Date'])
SG = SG.rename(columns={'Price': 'Close'})
SG = SG.replace(',','', regex=True)
SG['Close'] = SG['Close'].astype(float, errors = 'raise')
mask = (SG['Date'] >= start_date) & (SG['Date'] <= end_date)
SG = SG.loc[mask]
SG = SG.set_index('Date')
SG = SG.sort_index(axis=0, ascending = False)

In [63]:
def get_returns(df,d):
    '''
    The function obtains the log returns of the asset shifted d days
    
    PARAMETERS
    ----------
    df : pandas.DataFrame
        The data frame contains data of a chosen stock index 
        (Stock index must be arrange in DESCENDING ORDER by DATE)
    d : int
        The dth day being forecast
        (Assumed to be 1 for most cases)
        
    RETURNS
    -------
    df : pandas.DataFrame
        The data frame returns an updated data frame containing the
        'Returns' column
    '''
    df['Previous'] = df['Close'].shift(-d)
    df['Returns'] = np.log(df['Close']/df['Previous'])*100
    display(df)
    return df

SG = get_returns(SG, 1)

,Open,High,Low,Close,Previous,Returns
Date,,,,,,
2020-12-31,2853.60,2861.61,2843.81,2843.81,2869.22,-0.889551
2020-12-30,2855.06,2878.13,2846.22,2869.22,2848.14,0.737407
2020-12-29,2848.65,2852.65,2840.38,2848.14,2840.14,0.281280
2020-12-28,2845.85,2840.14,2840.14,2840.14,2842.04,-0.066876
2020-12-24,2840.65,2847.80,2834.14,2842.04,2833.40,0.304470
...,...,...,...,...,...,...
2006-01-09,2353.33,2353.33,2353.33,2353.33,2348.89,0.188847
2006-01-06,2348.89,2348.89,2348.89,2348.89,2334.08,0.632507
2006-01-05,2334.08,2334.08,2334.08,2334.08,2316.90,0.738772


# Diebold-Mariano Test

In [64]:
# Author   : John Tsang
# Date     : December 7th, 2017
# Purpose  : Implement the Diebold-Mariano Test (DM test) to compare 
#            forecast accuracy
# Input    : 1) actual_lst: the list of actual values
#            2) pred1_lst : the first list of predicted values
#            3) pred2_lst : the second list of predicted values
#            4) h         : the number of stpes ahead
#            5) crit      : a string specifying the criterion 
#                             i)  MSE : the mean squared error
#                            ii)  MAD : the mean absolute deviation
#                           iii) MAPE : the mean absolute percentage error
#                            iv) poly : use power function to weigh the errors
#            6) poly      : the power for crit power 
#                           (it is only meaningful when crit is "poly")
# Condition: 1) length of actual_lst, pred1_lst and pred2_lst is equal
#            2) h must be an integer and it must be greater than 0 and less than 
#               the length of actual_lst.
#            3) crit must take the 4 values specified in Input
#            4) Each value of actual_lst, pred1_lst and pred2_lst must
#               be numerical values. Missing values will not be accepted.
#            5) power must be a numerical value.
# Return   : a named-tuple of 2 elements
#            1) p_value : the p-value of the DM test
#            2) DM      : the test statistics of the DM test
##########################################################
# References:
#
# Harvey, D., Leybourne, S., & Newbold, P. (1997). Testing the equality of 
#   prediction mean squared errors. International Journal of forecasting, 
#   13(2), 281-291.
#
# Diebold, F. X. and Mariano, R. S. (1995), Comparing predictive accuracy, 
#   Journal of business & economic statistics 13(3), 253-264.
#
##########################################################
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

## Custom DM-Test Function

In [65]:
def DM_test(country, model_1, model_2, t):
    
    arr_model_1_dropped = model_1.drop(['Date'], axis = 1)
    arr_model_1_zero = arr_model_1_dropped.iloc[:, 0].to_numpy()

    arr_model_2_dropped = model_2.drop(['Date'], axis = 1)
    arr_model_2_zero = arr_model_2_dropped.iloc[:,0].to_numpy()

    returns = np.array(country['Returns'])[:t]

    dm = dm_test(returns, arr_model_1_zero, arr_model_2_zero, h = 1, crit = 'MSE')
    
    return dm

# Fixed Window

## 0.01%

In [66]:
SG_fw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/SG_VaR_FW-DN-0.01%25.csv')
SG_fw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/SG_VaR_FW-HS-0.01%25.csv')
SG_fw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/SG_VaR_FW-BRW-0.01%25.csv')
SG_fw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/SG_VaR_FW-G-0.01%25.csv')
SG_fw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/SG_VaR_FW-EA-0.01%25.csv')
SG_fw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/SG_VaR_FW-E-0.01%25.csv')
SG_fw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-FW/SG_fw_FW-I-0.01%25.csv')
SG_fw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-FW/SG_fw_FW-FI-0.01%25.csv')
SG_fw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-FW/SG_fw_FW-FI0.633-0.01%25.csv')

In [67]:
# DN
print(DM_test(SG, SG_fw_01_dn, SG_fw_01_garch, len(SG_fw_01_dn)))
print(DM_test(SG, SG_fw_01_dn, SG_fw_01_earch, len(SG_fw_01_dn)))
print(DM_test(SG, SG_fw_01_dn, SG_fw_01_egarch, len(SG_fw_01_dn)))
print(DM_test(SG, SG_fw_01_dn, SG_fw_01_igarch, len(SG_fw_01_dn)))
print(DM_test(SG, SG_fw_01_dn, SG_fw_01_figarch, len(SG_fw_01_dn)))
print(DM_test(SG, SG_fw_01_dn, SG_fw_01_figarch0633, len(SG_fw_01_dn)))

dm_return(DM=1.804273124860974, p_value=0.07156355984408663)
dm_return(DM=4.956428329559247, p_value=8.762923363439522e-07)
dm_return(DM=3.2804228612394795, p_value=0.0010812676916256187)
dm_return(DM=0.7823571033270135, p_value=0.4342356629764599)
dm_return(DM=1.4855751419784498, p_value=0.13778449469478268)
dm_return(DM=1.6610181555950354, p_value=0.09710064337978197)


In [68]:
# HS
print(DM_test(SG, SG_fw_01_hs, SG_fw_01_garch, len(SG_fw_01_hs)))
print(DM_test(SG, SG_fw_01_hs, SG_fw_01_earch, len(SG_fw_01_hs)))
print(DM_test(SG, SG_fw_01_hs, SG_fw_01_egarch, len(SG_fw_01_hs)))
print(DM_test(SG, SG_fw_01_hs, SG_fw_01_igarch, len(SG_fw_01_hs)))
print(DM_test(SG, SG_fw_01_hs, SG_fw_01_figarch, len(SG_fw_01_hs)))
print(DM_test(SG, SG_fw_01_hs, SG_fw_01_figarch0633, len(SG_fw_01_hs)))

dm_return(DM=26.82470397932163, p_value=1.2223185407022646e-113)
dm_return(DM=79.70538657076268, p_value=0.0)
dm_return(DM=31.65308886628966, p_value=2.5583689631917114e-143)
dm_return(DM=18.074487423612755, p_value=1.6201169788903783e-61)
dm_return(DM=26.55823802532868, p_value=5.311679007871897e-112)
dm_return(DM=26.912855658538785, p_value=3.508298952935813e-114)


In [69]:
# BRW
print(DM_test(SG, SG_fw_01_brw, SG_fw_01_garch, len(SG_fw_01_brw)))
print(DM_test(SG, SG_fw_01_brw, SG_fw_01_earch, len(SG_fw_01_brw)))
print(DM_test(SG, SG_fw_01_brw, SG_fw_01_egarch, len(SG_fw_01_brw)))
print(DM_test(SG, SG_fw_01_brw, SG_fw_01_igarch, len(SG_fw_01_brw)))
print(DM_test(SG, SG_fw_01_brw, SG_fw_01_figarch, len(SG_fw_01_brw)))
print(DM_test(SG, SG_fw_01_brw, SG_fw_01_figarch0633, len(SG_fw_01_brw)))

dm_return(DM=7.317110475163755, p_value=6.1557939040543e-13)
dm_return(DM=11.283438085997885, p_value=1.653545980621015e-27)
dm_return(DM=9.19602590233889, p_value=3.137613179193238e-19)
dm_return(DM=4.683381021315024, p_value=3.3132627257832254e-06)
dm_return(DM=7.058180057508426, p_value=3.653992418577364e-12)
dm_return(DM=7.237000323666274, p_value=1.0742379278126171e-12)


In [70]:
# GARCH
print(DM_test(SG, SG_fw_01_garch, SG_fw_01_earch, len(SG_fw_01_garch)))
print(DM_test(SG, SG_fw_01_garch, SG_fw_01_egarch, len(SG_fw_01_garch)))
print(DM_test(SG, SG_fw_01_garch, SG_fw_01_igarch, len(SG_fw_01_garch)))
print(DM_test(SG, SG_fw_01_garch, SG_fw_01_figarch, len(SG_fw_01_garch)))
print(DM_test(SG, SG_fw_01_garch, SG_fw_01_figarch0633, len(SG_fw_01_garch)))

dm_return(DM=-0.5628834515048626, p_value=0.573671563344833)
dm_return(DM=6.557934437996986, p_value=9.76952179263602e-11)
dm_return(DM=-0.7630492697385634, p_value=0.4456583955544239)
dm_return(DM=-2.589784215780369, p_value=0.009777817758509381)
dm_return(DM=-1.1422987381176928, p_value=0.25367077492938755)


In [71]:
# EARCH
print(DM_test(SG, SG_fw_01_earch, SG_fw_01_egarch, len(SG_fw_01_earch)))
print(DM_test(SG, SG_fw_01_earch, SG_fw_01_igarch, len(SG_fw_01_earch)))
print(DM_test(SG, SG_fw_01_earch, SG_fw_01_figarch, len(SG_fw_01_earch)))
print(DM_test(SG, SG_fw_01_earch, SG_fw_01_figarch0633, len(SG_fw_01_earch)))

dm_return(DM=2.0221423541695795, p_value=0.043493169372688066)
dm_return(DM=-0.1093761991461711, p_value=0.9129314714993481)
dm_return(DM=0.20764112095112094, p_value=0.8355619073316964)
dm_return(DM=0.3954193437234088, p_value=0.6926385896310063)


In [72]:
# EGARCH
print(DM_test(SG, SG_fw_01_egarch, SG_fw_01_igarch, len(SG_fw_01_egarch)))
print(DM_test(SG, SG_fw_01_egarch, SG_fw_01_figarch, len(SG_fw_01_egarch)))
print(DM_test(SG, SG_fw_01_egarch, SG_fw_01_figarch0633, len(SG_fw_01_egarch)))

dm_return(DM=-1.9070428122566372, p_value=0.056872077802918475)
dm_return(DM=-7.647890069497155, p_value=5.85104263088445e-14)
dm_return(DM=-6.681075436569096, p_value=4.4348385585101313e-11)


In [73]:
# IGARCH
print(DM_test(SG, SG_fw_01_igarch, SG_fw_01_figarch, len(SG_fw_01_igarch)))
print(DM_test(SG, SG_fw_01_igarch, SG_fw_01_figarch0633, len(SG_fw_01_igarch)))

dm_return(DM=0.41578876366560646, p_value=0.6776758095682931)
dm_return(DM=0.6261216804236351, p_value=0.531413284346365)


In [74]:
# FIGARCH
print(DM_test(SG, SG_fw_01_figarch, SG_fw_01_figarch0633, len(SG_fw_01_figarch)))

dm_return(DM=4.117788904519439, p_value=4.221770587433713e-05)


## 1.00 %

In [75]:
SG_fw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/SG_VaR_FW-DN-1%25.csv')
SG_fw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/SG_VaR_FW-HS-1%25.csv')
SG_fw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/SG_VaR_FW-BRW-1%25.csv')
SG_fw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/SG_VaR_FW-G-1%25.csv')
SG_fw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/SG_VaR_FW-EA-1%25.csv')
SG_fw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/SG_VaR_FW-E-1%25.csv')
SG_fw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-FW/SG_fw_FW-I-1%25.csv')
SG_fw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-FW/SG_fw_FW-FI-1%25.csv')
SG_fw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-FW/SG_fw_FW-FI0.633-1%25.csv')

In [76]:
# DN
print(DM_test(SG, SG_fw_1_dn, SG_fw_1_garch, len(SG_fw_1_dn)))
print(DM_test(SG, SG_fw_1_dn, SG_fw_1_earch, len(SG_fw_1_dn)))
print(DM_test(SG, SG_fw_1_dn, SG_fw_1_egarch, len(SG_fw_1_dn)))
print(DM_test(SG, SG_fw_1_dn, SG_fw_1_igarch, len(SG_fw_1_dn)))
print(DM_test(SG, SG_fw_1_dn, SG_fw_1_figarch, len(SG_fw_1_dn)))
print(DM_test(SG, SG_fw_1_dn, SG_fw_1_figarch0633, len(SG_fw_1_dn)))

dm_return(DM=1.8417220539466923, p_value=0.0658848963651165)
dm_return(DM=4.859256620809276, p_value=1.4173471701703027e-06)
dm_return(DM=3.3085962478132847, p_value=0.0009794560185787096)
dm_return(DM=0.7562799198764686, p_value=0.44970347072921546)
dm_return(DM=1.513740690774151, p_value=0.13048558922277562)
dm_return(DM=1.6788135009681202, p_value=0.0935779322230692)


In [77]:
# HS
print(DM_test(SG, SG_fw_1_hs, SG_fw_1_garch, len(SG_fw_1_hs)))
print(DM_test(SG, SG_fw_1_hs, SG_fw_1_earch, len(SG_fw_1_hs)))
print(DM_test(SG, SG_fw_1_hs, SG_fw_1_egarch, len(SG_fw_1_hs)))
print(DM_test(SG, SG_fw_1_hs, SG_fw_1_igarch, len(SG_fw_1_hs)))
print(DM_test(SG, SG_fw_1_hs, SG_fw_1_figarch, len(SG_fw_1_hs)))
print(DM_test(SG, SG_fw_1_hs, SG_fw_1_figarch0633, len(SG_fw_1_hs)))

dm_return(DM=9.295447021589181, p_value=1.3543255388036481e-19)
dm_return(DM=29.43109822470911, p_value=1.1093115460393768e-129)
dm_return(DM=11.801944688392803, p_value=9.400419688294293e-30)
dm_return(DM=5.734612276273224, p_value=1.384191103822098e-08)
dm_return(DM=8.969911087892362, p_value=2.0649130993402476e-18)
dm_return(DM=9.181740606983263, p_value=3.538123945856749e-19)


In [78]:
# BRW
print(DM_test(SG, SG_fw_1_brw, SG_fw_1_garch, len(SG_fw_1_brw)))
print(DM_test(SG, SG_fw_1_brw, SG_fw_1_earch, len(SG_fw_1_brw)))
print(DM_test(SG, SG_fw_1_brw, SG_fw_1_egarch, len(SG_fw_1_brw)))
print(DM_test(SG, SG_fw_1_brw, SG_fw_1_igarch, len(SG_fw_1_brw)))
print(DM_test(SG, SG_fw_1_brw, SG_fw_1_figarch, len(SG_fw_1_brw)))
print(DM_test(SG, SG_fw_1_brw, SG_fw_1_figarch0633, len(SG_fw_1_brw)))

dm_return(DM=8.356439592008728, p_value=2.835729544018066e-16)
dm_return(DM=9.018040003153923, p_value=1.3869980605588328e-18)
dm_return(DM=10.161141487443755, p_value=6.71476811342268e-23)
dm_return(DM=4.766414013555803, p_value=2.2265244946357722e-06)
dm_return(DM=7.94169379275638, p_value=6.729494239091475e-15)
dm_return(DM=8.074582748606506, p_value=2.474952021573335e-15)


In [79]:
# GARCH
print(DM_test(SG, SG_fw_1_garch, SG_fw_1_earch, len(SG_fw_1_garch)))
print(DM_test(SG, SG_fw_1_garch, SG_fw_1_egarch, len(SG_fw_1_garch)))
print(DM_test(SG, SG_fw_1_garch, SG_fw_1_igarch, len(SG_fw_1_garch)))
print(DM_test(SG, SG_fw_1_garch, SG_fw_1_figarch, len(SG_fw_1_garch)))
print(DM_test(SG, SG_fw_1_garch, SG_fw_1_figarch0633, len(SG_fw_1_garch)))

dm_return(DM=-0.6146884665323022, p_value=0.5389348268110085)
dm_return(DM=6.552535916492718, p_value=1.0110794686901096e-10)
dm_return(DM=-0.8389405872208245, p_value=0.4017526329374743)
dm_return(DM=-2.662938667301186, p_value=0.007901436374504835)
dm_return(DM=-1.2674829426094574, p_value=0.2053505452326459)


In [80]:
# EARCH
print(DM_test(SG, SG_fw_1_earch, SG_fw_1_egarch, len(SG_fw_1_earch)))
print(DM_test(SG, SG_fw_1_earch, SG_fw_1_igarch, len(SG_fw_1_earch)))
print(DM_test(SG, SG_fw_1_earch, SG_fw_1_figarch, len(SG_fw_1_earch)))
print(DM_test(SG, SG_fw_1_earch, SG_fw_1_figarch0633, len(SG_fw_1_earch)))

dm_return(DM=2.080560523256807, p_value=0.03779103907793528)
dm_return(DM=-0.12657675041551092, p_value=0.8993071253646135)
dm_return(DM=0.24802261569819173, p_value=0.8041803970019799)
dm_return(DM=0.42627864556675754, p_value=0.6700192027090479)


In [81]:
# EGARCH
print(DM_test(SG, SG_fw_1_egarch, SG_fw_1_igarch, len(SG_fw_1_egarch)))
print(DM_test(SG, SG_fw_1_egarch, SG_fw_1_figarch, len(SG_fw_1_egarch)))
print(DM_test(SG, SG_fw_1_egarch, SG_fw_1_figarch0633, len(SG_fw_1_egarch)))

dm_return(DM=-1.9691185132216487, p_value=0.049283368332433485)
dm_return(DM=-7.690380822683084, p_value=4.297483132794747e-14)
dm_return(DM=-6.7405999265541805, p_value=3.013879096757207e-11)


In [82]:
# IGARCH
print(DM_test(SG, SG_fw_1_igarch, SG_fw_1_figarch, len(SG_fw_1_igarch)))
print(DM_test(SG, SG_fw_1_igarch, SG_fw_1_figarch0633, len(SG_fw_1_igarch)))

dm_return(DM=0.48759164060313426, p_value=0.6259723731825253)
dm_return(DM=0.6860925375215476, p_value=0.49285304691867504)


In [83]:
# FIGARCH
print(DM_test(SG, SG_fw_1_figarch, SG_fw_1_figarch0633, len(SG_fw_1_figarch)))

dm_return(DM=3.6652739725058656, p_value=0.0002633384559674778)


## 5.00 %

In [84]:
SG_fw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/SG_VaR_FW-DN-5%25.csv')
SG_fw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/SG_VaR_FW-HS-5%25.csv')
SG_fw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/SG_VaR_FW-BRW-5%25.csv')
SG_fw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/SG_VaR_FW-G-5%25.csv')
SG_fw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/SG_VaR_FW-EA-5%25.csv')
SG_fw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/SG_VaR_FW-E-5%25.csv')
SG_fw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-FW/SG_fw_FW-I-5%25.csv')
SG_fw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-FW/SG_fw_FW-FI-5%25.csv')
SG_fw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-FW/SG_fw_FW-FI0.633-5%25.csv')

In [85]:
# DN
print(DM_test(SG, SG_fw_5_dn, SG_fw_5_garch, len(SG_fw_5_dn)))
print(DM_test(SG, SG_fw_5_dn, SG_fw_5_earch, len(SG_fw_5_dn)))
print(DM_test(SG, SG_fw_5_dn, SG_fw_5_egarch, len(SG_fw_5_dn)))
print(DM_test(SG, SG_fw_5_dn, SG_fw_5_igarch, len(SG_fw_5_dn)))
print(DM_test(SG, SG_fw_5_dn, SG_fw_5_figarch, len(SG_fw_5_dn)))
print(DM_test(SG, SG_fw_5_dn, SG_fw_5_figarch0633, len(SG_fw_5_dn)))

dm_return(DM=1.8701475414571709, p_value=0.0618274121291929)
dm_return(DM=4.7107932485556665, p_value=2.9077520051994214e-06)
dm_return(DM=3.3118035340629612, p_value=0.0009684474473965157)
dm_return(DM=0.7243719722694564, p_value=0.46904865857735845)
dm_return(DM=1.534456082428784, p_value=0.1253118677267297)
dm_return(DM=1.6863186783571598, p_value=0.09212336180423832)


In [86]:
# HS
print(DM_test(SG, SG_fw_5_hs, SG_fw_5_garch, len(SG_fw_5_hs)))
print(DM_test(SG, SG_fw_5_hs, SG_fw_5_earch, len(SG_fw_5_hs)))
print(DM_test(SG, SG_fw_5_hs, SG_fw_5_egarch, len(SG_fw_5_hs)))
print(DM_test(SG, SG_fw_5_hs, SG_fw_5_igarch, len(SG_fw_5_hs)))
print(DM_test(SG, SG_fw_5_hs, SG_fw_5_figarch, len(SG_fw_5_hs)))
print(DM_test(SG, SG_fw_5_hs, SG_fw_5_figarch0633, len(SG_fw_5_hs)))

dm_return(DM=-2.439992025639953, p_value=0.01490318181698322)
dm_return(DM=-11.108878508870431, p_value=9.082678269345405e-27)
dm_return(DM=-1.6083735955093583, p_value=0.10814693884370515)
dm_return(DM=-2.1294978903839015, p_value=0.033517050317876065)
dm_return(DM=-2.7751225579398695, p_value=0.00564650532224152)
dm_return(DM=-2.6525419651754265, p_value=0.008146729492392901)


In [87]:
# BRW
print(DM_test(SG, SG_fw_5_brw, SG_fw_5_garch, len(SG_fw_5_brw)))
print(DM_test(SG, SG_fw_5_brw, SG_fw_5_earch, len(SG_fw_5_brw)))
print(DM_test(SG, SG_fw_5_brw, SG_fw_5_egarch, len(SG_fw_5_brw)))
print(DM_test(SG, SG_fw_5_brw, SG_fw_5_igarch, len(SG_fw_5_brw)))
print(DM_test(SG, SG_fw_5_brw, SG_fw_5_figarch, len(SG_fw_5_brw)))
print(DM_test(SG, SG_fw_5_brw, SG_fw_5_figarch0633, len(SG_fw_5_brw)))

dm_return(DM=-4.3862000106070616, p_value=1.3073242351720654e-05)
dm_return(DM=-14.279526604140129, p_value=2.2018188979491933e-41)
dm_return(DM=-3.797009857501774, p_value=0.0001575016394857676)
dm_return(DM=-3.3129495600297143, p_value=0.000964541788732489)
dm_return(DM=-4.722552529300304, p_value=2.7488212089794493e-06)
dm_return(DM=-4.600509540199556, p_value=4.896765794157834e-06)


In [88]:
# GARCH
print(DM_test(SG, SG_fw_5_garch, SG_fw_5_earch, len(SG_fw_5_garch)))
print(DM_test(SG, SG_fw_5_garch, SG_fw_5_egarch, len(SG_fw_5_garch)))
print(DM_test(SG, SG_fw_5_garch, SG_fw_5_igarch, len(SG_fw_5_garch)))
print(DM_test(SG, SG_fw_5_garch, SG_fw_5_figarch, len(SG_fw_5_garch)))
print(DM_test(SG, SG_fw_5_garch, SG_fw_5_figarch0633, len(SG_fw_5_garch)))

dm_return(DM=-0.6685969578455676, p_value=0.5039450009478386)
dm_return(DM=6.521852754315724, p_value=1.2283992055420914e-10)
dm_return(DM=-0.9189113104513467, p_value=0.358418242457318)
dm_return(DM=-2.7330922826697144, p_value=0.006412428335808843)
dm_return(DM=-1.3962945869652317, p_value=0.1630121362324885)


In [89]:
# EARCH
print(DM_test(SG, SG_fw_5_earch, SG_fw_5_egarch, len(SG_fw_5_earch)))
print(DM_test(SG, SG_fw_5_earch, SG_fw_5_igarch, len(SG_fw_5_earch)))
print(DM_test(SG, SG_fw_5_earch, SG_fw_5_figarch, len(SG_fw_5_earch)))
print(DM_test(SG, SG_fw_5_earch, SG_fw_5_figarch0633, len(SG_fw_5_earch)))

dm_return(DM=2.130956285353266, p_value=0.03339631157215439)
dm_return(DM=-0.14461413783526852, p_value=0.8850518666993248)
dm_return(DM=0.2913137431889268, p_value=0.7708867029249532)
dm_return(DM=0.4578831458527365, p_value=0.6471604229256137)


In [90]:
# EGARCH
print(DM_test(SG, SG_fw_5_egarch, SG_fw_5_igarch, len(SG_fw_5_egarch)))
print(DM_test(SG, SG_fw_5_egarch, SG_fw_5_figarch, len(SG_fw_5_egarch)))
print(DM_test(SG, SG_fw_5_egarch, SG_fw_5_figarch0633, len(SG_fw_5_egarch)))

dm_return(DM=-2.0313676274718286, p_value=0.042546992413397)
dm_return(DM=-7.708307474053117, p_value=3.771264397664913e-14)
dm_return(DM=-6.774455364808455, p_value=2.416285763943673e-11)


In [91]:
# IGARCH
print(DM_test(SG, SG_fw_5_igarch, SG_fw_5_figarch,  len(SG_fw_5_igarch)))
print(DM_test(SG, SG_fw_5_igarch, SG_fw_5_figarch0633, len(SG_fw_5_igarch)))

dm_return(DM=0.5639296325848933, p_value=0.5729596766809233)
dm_return(DM=0.7494515626648943, p_value=0.45380485132789694)


In [92]:
# FIGARCH
print(DM_test(SG, SG_fw_5_figarch, SG_fw_5_figarch0633, len(SG_fw_5_figarch)))

dm_return(DM=3.1838228672435247, p_value=0.0015095430160510417)


# Rolling Window

## 0.01 %

In [93]:
SG_rw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/SG_VaR_RW-DN-0.01%25.csv')
SG_rw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/SG_VaR_RW-HS-0.01%25.csv')
SG_rw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/SG_VaR_RW-BRW-0.01%25.csv')
SG_rw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/SG_VaR_RW-G-0.01%25.csv')
SG_rw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/SG_VaR_RW-EA-0.01%25.csv')
SG_rw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/SG_VaR_RW-E-0.01%25.csv')
SG_rw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-RW/SG_RW-I-0.01%25.csv')
SG_rw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-RW/SG_RW-FI-0.01%25.csv')
SG_rw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-RW/SG_RW-FI0.633-0.01%25.csv')

In [94]:
# DN
print(DM_test(SG, SG_rw_01_dn, SG_rw_01_garch, len(SG_rw_01_dn)))
print(DM_test(SG, SG_rw_01_dn, SG_rw_01_earch, len(SG_rw_01_dn)))
print(DM_test(SG, SG_rw_01_dn, SG_rw_01_egarch, len(SG_rw_01_dn)))
print(DM_test(SG, SG_rw_01_dn, SG_rw_01_igarch, len(SG_rw_01_dn)))
print(DM_test(SG, SG_rw_01_dn, SG_rw_01_figarch, len(SG_rw_01_dn)))
print(DM_test(SG, SG_rw_01_dn, SG_rw_01_figarch0633, len(SG_rw_01_dn)))

dm_return(DM=2.300704913118407, p_value=0.021664063540607535)
dm_return(DM=5.4524798556546425, p_value=6.617400550068555e-08)
dm_return(DM=3.9053149789282835, p_value=0.00010202385349820028)
dm_return(DM=1.1845770865786838, p_value=0.23653562368437814)
dm_return(DM=1.974818215710921, p_value=0.04863138400580271)
dm_return(DM=2.2638059017101106, p_value=0.02385243440003817)


In [95]:
# HS
print(DM_test(SG, SG_rw_01_hs, SG_rw_01_garch, len(SG_rw_01_hs)))
print(DM_test(SG, SG_rw_01_hs, SG_rw_01_earch, len(SG_rw_01_hs)))
print(DM_test(SG, SG_rw_01_hs, SG_rw_01_egarch, len(SG_rw_01_hs)))
print(DM_test(SG, SG_rw_01_hs, SG_rw_01_igarch, len(SG_rw_01_hs)))
print(DM_test(SG, SG_rw_01_hs, SG_rw_01_figarch, len(SG_rw_01_hs)))
print(DM_test(SG, SG_rw_01_hs, SG_rw_01_figarch0633, len(SG_rw_01_hs)))

dm_return(DM=43.21321357080465, p_value=1.2891569121876108e-211)
dm_return(DM=115.79099780819979, p_value=0.0)
dm_return(DM=50.13795022231656, p_value=2.1229426041363515e-249)
dm_return(DM=30.74045039833181, p_value=9.991332139839e-138)
dm_return(DM=42.64152707157704, p_value=2.2079918941439366e-208)
dm_return(DM=44.91940334936369, p_value=3.643097399573297e-221)


In [96]:
# BRW
print(DM_test(SG, SG_rw_01_brw, SG_rw_01_garch, len(SG_rw_01_brw)))
print(DM_test(SG, SG_rw_01_brw, SG_rw_01_earch, len(SG_rw_01_brw)))
print(DM_test(SG, SG_rw_01_brw, SG_rw_01_egarch, len(SG_rw_01_brw)))
print(DM_test(SG, SG_rw_01_brw, SG_rw_01_igarch, len(SG_rw_01_brw)))
print(DM_test(SG, SG_rw_01_brw, SG_rw_01_figarch, len(SG_rw_01_brw)))
print(DM_test(SG, SG_rw_01_brw, SG_rw_01_figarch0633, len(SG_rw_01_brw)))

dm_return(DM=7.1299498067818385, p_value=2.242498742214341e-12)
dm_return(DM=10.904239688530268, p_value=6.530052825960898e-26)
dm_return(DM=9.069368877527259, p_value=9.056321815824643e-19)
dm_return(DM=4.699424030784283, p_value=3.069785746486909e-06)
dm_return(DM=6.823687026806921, p_value=1.7492402350778395e-11)
dm_return(DM=7.329501808129743, p_value=5.645213036433455e-13)


In [97]:
# GARCH
print(DM_test(SG, SG_rw_01_garch, SG_rw_01_earch, len(SG_rw_01_garch)))
print(DM_test(SG, SG_rw_01_garch, SG_rw_01_egarch, len(SG_rw_01_garch)))
print(DM_test(SG, SG_rw_01_garch, SG_rw_01_igarch, len(SG_rw_01_garch)))
print(DM_test(SG, SG_rw_01_garch, SG_rw_01_figarch, len(SG_rw_01_garch)))
print(DM_test(SG, SG_rw_01_garch, SG_rw_01_figarch0633, len(SG_rw_01_garch)))

dm_return(DM=-1.0352644343849382, p_value=0.30085748737894263)
dm_return(DM=6.6683531790790145, p_value=4.8146736294725195e-11)
dm_return(DM=-0.7919002841103172, p_value=0.4286529769608838)
dm_return(DM=-3.089657204777975, p_value=0.0020730451236334268)
dm_return(DM=-1.3033208500941067, p_value=0.1928392741793069)


In [98]:
# EARCH
print(DM_test(SG, SG_rw_01_earch, SG_rw_01_egarch, len(SG_rw_01_earch)))
print(DM_test(SG, SG_rw_01_earch, SG_rw_01_igarch, len(SG_rw_01_earch)))
print(DM_test(SG, SG_rw_01_earch, SG_rw_01_figarch, len(SG_rw_01_earch)))
print(DM_test(SG, SG_rw_01_earch, SG_rw_01_figarch0633, len(SG_rw_01_earch)))

dm_return(DM=2.6282795140178803, p_value=0.008745919448573542)
dm_return(DM=0.25899576929098767, p_value=0.7957049643523283)
dm_return(DM=0.6869044007792556, p_value=0.49234153839943096)
dm_return(DM=0.9189336898704407, p_value=0.3584065432002902)


In [99]:
# EGARCH
print(DM_test(SG, SG_rw_01_egarch, SG_rw_01_igarch, len(SG_rw_01_egarch)))
print(DM_test(SG, SG_rw_01_egarch, SG_rw_01_figarch, len(SG_rw_01_egarch)))
print(DM_test(SG, SG_rw_01_egarch, SG_rw_01_figarch0633, len(SG_rw_01_egarch)))

dm_return(DM=-2.0793397637147506, p_value=0.037903336704102654)
dm_return(DM=-7.783974307271751, p_value=2.166898383810682e-14)
dm_return(DM=-8.419591261661772, p_value=1.7308009646757494e-16)


In [100]:
# IGARCH
print(DM_test(SG, SG_rw_01_igarch, SG_rw_01_figarch,  len(SG_rw_01_igarch)))
print(DM_test(SG, SG_rw_01_igarch, SG_rw_01_figarch0633, len(SG_rw_01_igarch)))

dm_return(DM=0.40736333238228034, p_value=0.6838498597885011)
dm_return(DM=0.5756212363716647, p_value=0.5650326936881976)


In [101]:
# FIGARCH
print(DM_test(SG, SG_rw_01_figarch, SG_rw_01_figarch0633, len(SG_rw_01_figarch)))

dm_return(DM=1.8117375096386978, p_value=0.07040068917849095)


## 1.00 %

In [102]:
SG_rw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/SG_VaR_RW-DN-1%25.csv')
SG_rw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/SG_VaR_RW-HS-1%25.csv')
SG_rw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/SG_VaR_RW-BRW-1%25.csv')
SG_rw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/SG_VaR_RW-G-1%25.csv')
SG_rw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/SG_VaR_RW-EA-1%25.csv')
SG_rw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/SG_VaR_RW-E-1%25.csv')
SG_rw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-RW/SG_RW-I-1%25.csv')
SG_rw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-RW/SG_RW-FI-1%25.csv')
SG_rw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-RW/SG_RW-FI0.633-1%25.csv')

In [103]:
# DN
print(DM_test(SG, SG_rw_1_dn, SG_rw_1_garch, len(SG_rw_1_dn)))
print(DM_test(SG, SG_rw_1_dn, SG_rw_1_earch, len(SG_rw_1_dn)))
print(DM_test(SG, SG_rw_1_dn, SG_rw_1_egarch, len(SG_rw_1_dn)))
print(DM_test(SG, SG_rw_1_dn, SG_rw_1_igarch, len(SG_rw_1_dn)))
print(DM_test(SG, SG_rw_1_dn, SG_rw_1_figarch, len(SG_rw_1_dn)))
print(DM_test(SG, SG_rw_1_dn, SG_rw_1_figarch0633, len(SG_rw_1_dn)))

dm_return(DM=2.325793190571685, p_value=0.020278077879158418)
dm_return(DM=5.339123691038378, p_value=1.2169746605730528e-07)
dm_return(DM=3.912977875244339, p_value=9.889768655513909e-05)
dm_return(DM=1.153445834316657, p_value=0.24907107056839178)
dm_return(DM=1.9963819046595273, p_value=0.04623001661046777)
dm_return(DM=2.2883906064681585, p_value=0.022374086598821775)


In [104]:
# HS
print(DM_test(SG, SG_rw_1_hs, SG_rw_1_garch, len(SG_rw_1_hs)))
print(DM_test(SG, SG_rw_1_hs, SG_rw_1_earch, len(SG_rw_1_hs)))
print(DM_test(SG, SG_rw_1_hs, SG_rw_1_egarch, len(SG_rw_1_hs)))
print(DM_test(SG, SG_rw_1_hs, SG_rw_1_igarch, len(SG_rw_1_hs)))
print(DM_test(SG, SG_rw_1_hs, SG_rw_1_figarch, len(SG_rw_1_hs)))
print(DM_test(SG, SG_rw_1_hs, SG_rw_1_figarch0633, len(SG_rw_1_hs)))

dm_return(DM=10.315369294623856, p_value=1.6401174952536586e-23)
dm_return(DM=32.14493207438005, p_value=2.522003962513647e-146)
dm_return(DM=12.994194155257881, p_value=3.573512739716713e-35)
dm_return(DM=6.747738926411004, p_value=2.876877942802823e-11)
dm_return(DM=9.939105527760598, p_value=4.966893508909072e-22)
dm_return(DM=10.668502509523293, p_value=6.129638086228242e-25)


In [105]:
# BRW
print(DM_test(SG, SG_rw_1_brw, SG_rw_1_garch, len(SG_rw_1_brw)))
print(DM_test(SG, SG_rw_1_brw, SG_rw_1_earch, len(SG_rw_1_brw)))
print(DM_test(SG, SG_rw_1_brw, SG_rw_1_egarch, len(SG_rw_1_brw)))
print(DM_test(SG, SG_rw_1_brw, SG_rw_1_igarch, len(SG_rw_1_brw)))
print(DM_test(SG, SG_rw_1_brw, SG_rw_1_figarch, len(SG_rw_1_brw)))
print(DM_test(SG, SG_rw_1_brw, SG_rw_1_figarch0633, len(SG_rw_1_brw)))

dm_return(DM=8.100501465134286, p_value=2.033049555578639e-15)
dm_return(DM=8.330617960105771, p_value=3.4670368206984995e-16)
dm_return(DM=9.967777966236381, p_value=3.8430817078460666e-22)
dm_return(DM=4.80931372705121, p_value=1.8088480990482525e-06)
dm_return(DM=7.700537618053752, p_value=3.991052477808711e-14)
dm_return(DM=8.37678326719064, p_value=2.4195549361066046e-16)


In [106]:
# GARCH
print(DM_test(SG, SG_rw_1_garch, SG_rw_1_earch, len(SG_rw_1_garch)))
print(DM_test(SG, SG_rw_1_garch, SG_rw_1_egarch, len(SG_rw_1_garch)))
print(DM_test(SG, SG_rw_1_garch, SG_rw_1_igarch, len(SG_rw_1_garch)))
print(DM_test(SG, SG_rw_1_garch, SG_rw_1_figarch, len(SG_rw_1_garch)))
print(DM_test(SG, SG_rw_1_garch, SG_rw_1_figarch0633, len(SG_rw_1_garch)))

dm_return(DM=-1.078799404246468, p_value=0.28100150450162437)
dm_return(DM=6.661294959117553, p_value=5.0389704896834355e-11)
dm_return(DM=-0.8649171475050978, p_value=0.3873428938575093)
dm_return(DM=-3.137132106776064, p_value=0.001768418055592406)
dm_return(DM=-1.3385401084192323, p_value=0.18109975407932222)


In [107]:
# EARCH
print(DM_test(SG, SG_rw_1_earch, SG_rw_1_egarch, len(SG_rw_1_earch)))
print(DM_test(SG, SG_rw_1_earch, SG_rw_1_igarch, len(SG_rw_1_earch)))
print(DM_test(SG, SG_rw_1_earch, SG_rw_1_figarch, len(SG_rw_1_earch)))
print(DM_test(SG, SG_rw_1_earch, SG_rw_1_figarch0633, len(SG_rw_1_earch)))

dm_return(DM=2.6719654125991403, p_value=0.007693862588970247)
dm_return(DM=0.23906100920196308, p_value=0.8111193513729121)
dm_return(DM=0.7242001530727984, p_value=0.46915406286550343)
dm_return(DM=0.9602455101188739, p_value=0.33722106841046895)


In [108]:
# EGARCH
print(DM_test(SG, SG_rw_1_egarch, SG_rw_1_igarch, len(SG_rw_1_egarch)))
print(DM_test(SG, SG_rw_1_egarch, SG_rw_1_figarch, len(SG_rw_1_egarch)))
print(DM_test(SG, SG_rw_1_egarch, SG_rw_1_figarch0633, len(SG_rw_1_egarch)))

dm_return(DM=-2.135658527398848, p_value=0.03300955430143078)
dm_return(DM=-7.82310069918174, p_value=1.6241941827385672e-14)
dm_return(DM=-8.464733923628339, p_value=1.2138715678284338e-16)


In [109]:
# IGARCH
print(DM_test(SG, SG_rw_1_igarch, SG_rw_1_figarch,  len(SG_rw_1_igarch)))
print(DM_test(SG, SG_rw_1_igarch, SG_rw_1_figarch0633, len(SG_rw_1_igarch)))

dm_return(DM=0.4798241365039791, p_value=0.6314832759119078)
dm_return(DM=0.6397223248189909, p_value=0.5225358314478474)


In [110]:
# FIGARCH
print(DM_test(SG, SG_rw_1_figarch, SG_rw_1_figarch0633, len(SG_rw_1_figarch)))

dm_return(DM=1.8015796161371571, p_value=0.07198703217941273)


## 5.00 %

In [111]:
SG_rw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/SG_VaR_RW-DN-5%25.csv')
SG_rw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/SG_VaR_RW-HS-5%25.csv')
SG_rw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/SG_VaR_RW-BRW-5%25.csv')
SG_rw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/SG_VaR_RW-G-5%25.csv')
SG_rw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/SG_VaR_RW-EA-5%25.csv')
SG_rw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/SG_VaR_RW-E-5%25.csv')
SG_rw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-RW/SG_RW-I-5%25.csv')
SG_rw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-RW/SG_RW-FI-5%25.csv')
SG_rw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-RW/SG_RW-FI0.633-5%25.csv')

In [112]:
# DN
print(DM_test(SG, SG_rw_5_dn, SG_rw_5_garch, len(SG_rw_5_dn)))
print(DM_test(SG, SG_rw_5_dn, SG_rw_5_earch, len(SG_rw_5_dn)))
print(DM_test(SG, SG_rw_5_dn, SG_rw_5_egarch, len(SG_rw_5_dn)))
print(DM_test(SG, SG_rw_5_dn, SG_rw_5_igarch, len(SG_rw_5_dn)))
print(DM_test(SG, SG_rw_5_dn, SG_rw_5_figarch, len(SG_rw_5_dn)))
print(DM_test(SG, SG_rw_5_dn, SG_rw_5_figarch0633, len(SG_rw_5_dn)))

dm_return(DM=2.3362707414537596, p_value=0.01972259470173544)
dm_return(DM=5.167698357140767, p_value=2.9936628592356776e-07)
dm_return(DM=3.8860769193587585, p_value=0.00011028911466200968)
dm_return(DM=1.1136369524385183, p_value=0.26576861370584015)
dm_return(DM=2.005799261140184, p_value=0.045213102969727505)
dm_return(DM=2.297481909608737, p_value=0.021847972211995224)


In [113]:
# HS
print(DM_test(SG, SG_rw_5_hs, SG_rw_5_garch, len(SG_rw_5_hs)))
print(DM_test(SG, SG_rw_5_hs, SG_rw_5_earch, len(SG_rw_5_hs)))
print(DM_test(SG, SG_rw_5_hs, SG_rw_5_egarch, len(SG_rw_5_hs)))
print(DM_test(SG, SG_rw_5_hs, SG_rw_5_igarch, len(SG_rw_5_hs)))
print(DM_test(SG, SG_rw_5_hs, SG_rw_5_figarch, len(SG_rw_5_hs)))
print(DM_test(SG, SG_rw_5_hs, SG_rw_5_figarch0633, len(SG_rw_5_hs)))

dm_return(DM=-1.9043563340882064, p_value=0.05722135944765169)
dm_return(DM=-11.153460442599853, p_value=5.889201382094409e-27)
dm_return(DM=-0.9603278528131396, p_value=0.3371796634080795)
dm_return(DM=-1.7968197913355035, p_value=0.07274039657282137)
dm_return(DM=-2.2068948907407826, p_value=0.027603988789978873)
dm_return(DM=-2.1632907658232137, p_value=0.030813570031118898)


In [114]:
# BRW
print(DM_test(SG, SG_rw_5_brw, SG_rw_5_garch, len(SG_rw_5_brw)))
print(DM_test(SG, SG_rw_5_brw, SG_rw_5_earch, len(SG_rw_5_brw)))
print(DM_test(SG, SG_rw_5_brw, SG_rw_5_egarch, len(SG_rw_5_brw)))
print(DM_test(SG, SG_rw_5_brw, SG_rw_5_igarch, len(SG_rw_5_brw)))
print(DM_test(SG, SG_rw_5_brw, SG_rw_5_figarch, len(SG_rw_5_brw)))
print(DM_test(SG, SG_rw_5_brw, SG_rw_5_figarch0633, len(SG_rw_5_brw)))

dm_return(DM=-4.5173807139201525, p_value=7.201717015390217e-06)
dm_return(DM=-16.35102007606562, p_value=4.4159746032216355e-52)
dm_return(DM=-3.9221552889505973, p_value=9.52728689593079e-05)
dm_return(DM=-3.4746186245198665, p_value=0.0005390325494425054)
dm_return(DM=-4.804595045666196, p_value=1.8508055084648604e-06)
dm_return(DM=-4.948968289484491, p_value=9.095127484892449e-07)


In [115]:
# GARCH
print(DM_test(SG, SG_rw_5_garch, SG_rw_5_earch, len(SG_rw_5_garch)))
print(DM_test(SG, SG_rw_5_garch, SG_rw_5_egarch, len(SG_rw_5_garch)))
print(DM_test(SG, SG_rw_5_garch, SG_rw_5_igarch, len(SG_rw_5_garch)))
print(DM_test(SG, SG_rw_5_garch, SG_rw_5_figarch, len(SG_rw_5_garch)))
print(DM_test(SG, SG_rw_5_garch, SG_rw_5_figarch0633, len(SG_rw_5_garch)))

dm_return(DM=-1.120421950775595, p_value=0.262869394177642)
dm_return(DM=6.6292562069746275, p_value=6.192691769322058e-11)
dm_return(DM=-0.9417258074226227, p_value=0.34661660893067625)
dm_return(DM=-3.1770443863417674, p_value=0.0015448190919892576)
dm_return(DM=-1.3714825529119001, p_value=0.17060815590500636)


In [116]:
# EARCH
print(DM_test(SG, SG_rw_5_earch, SG_rw_5_egarch, len(SG_rw_5_earch)))
print(DM_test(SG, SG_rw_5_earch, SG_rw_5_igarch, len(SG_rw_5_earch)))
print(DM_test(SG, SG_rw_5_earch, SG_rw_5_figarch, len(SG_rw_5_earch)))
print(DM_test(SG, SG_rw_5_earch, SG_rw_5_figarch0633, len(SG_rw_5_earch)))

dm_return(DM=2.700110550160404, p_value=0.0070777186238273945)
dm_return(DM=0.21640847644751637, p_value=0.828724391845869)
dm_return(DM=0.7611749907270599, p_value=0.4467763007896034)
dm_return(DM=0.9999171262201078, p_value=0.31765220565438285)


In [117]:
# EGARCH
print(DM_test(SG, SG_rw_5_egarch, SG_rw_5_igarch, len(SG_rw_5_egarch)))
print(DM_test(SG, SG_rw_5_egarch, SG_rw_5_figarch, len(SG_rw_5_egarch)))
print(DM_test(SG, SG_rw_5_egarch, SG_rw_5_figarch0633, len(SG_rw_5_egarch)))

dm_return(DM=-2.1914983021780863, p_value=0.02870266264136473)
dm_return(DM=-7.838577351047989, p_value=1.4486742260882123e-14)
dm_return(DM=-8.464688871823046, p_value=1.2143023520348484e-16)


In [118]:
# IGARCH
print(DM_test(SG, SG_rw_5_igarch, SG_rw_5_figarch,  len(SG_rw_5_igarch)))
print(DM_test(SG, SG_rw_5_igarch, SG_rw_5_figarch0633, len(SG_rw_5_igarch)))

dm_return(DM=0.5569479724208265, p_value=0.5777183618648273)
dm_return(DM=0.7076460211431662, p_value=0.47937069088840045)


In [119]:
# FIGARCH
print(DM_test(SG, SG_rw_5_figarch, SG_rw_5_figarch0633, len(SG_rw_5_figarch)))

dm_return(DM=1.7888250552292988, p_value=0.07402030945186697)
